In [24]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [2]:
#read in incidence csv from github

incidence_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Age_Incidence_Cancer_DB.csv")                         
#get rid of commas in columns
incidence_df = incidence_df.replace(',','', regex=True)
incidence_df.head()     

,average_annual_count,recent_trend,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,five_year_trend_rates,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019
0,9,stable,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,167.8,2.3,Lowndes County AL,54.0,10350,10248,10097,9974,9726
1,72,rising,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,50+,1738.4,2.0,Lowndes County AL,54.0,10350,10248,10097,9974,9726
2,37,rising,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<65,339.4,2.4,Lowndes County AL,54.0,10350,10248,10097,9974,9726
3,44,stable,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,65+,2416.0,1.8,Lowndes County AL,54.0,10350,10248,10097,9974,9726
4,9,stable,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,145.2,2.0,Conecuh County AL,51.0,12678,12500,12431,12292,12067


In [3]:
#check dtypes
incidence_df.dtypes

average_annual_count                     int64
recent_trend                            object
primary_state_name                      object
cancer                                  object
race_ethnicity                          object
sex                                     object
age                                     object
age_adjusted_rate                       object
five_year_trend_rates                   object
county_state                            object
index_of_medical_underservice_score    float64
popestimate2015                          int64
popestimate2016                          int64
popestimate2017                          int64
popestimate2018                          int64
popestimate2019                          int64
dtype: object

In [4]:
#get count of rows
incidence_df.count()

average_annual_count                   10229
recent_trend                           10229
primary_state_name                     10229
cancer                                 10229
race_ethnicity                         10229
sex                                    10229
age                                    10229
age_adjusted_rate                      10229
five_year_trend_rates                  10229
county_state                           10229
index_of_medical_underservice_score     8793
popestimate2015                        10229
popestimate2016                        10229
popestimate2017                        10229
popestimate2018                        10229
popestimate2019                        10229
dtype: int64

In [5]:
#get count of values for age
incidence_df["age"].value_counts()

50+    2643
65+    2628
<65    2604
<50    2354
Name: age, dtype: int64

In [6]:
#get count of NA values for MUA
incidence_df["index_of_medical_underservice_score"].isna().sum()

1436

In [7]:
#drop rows where MUA index is null
incidence_df = incidence_df[incidence_df["index_of_medical_underservice_score"].isna() == False]

In [8]:
#get count of rows
incidence_df.count()

average_annual_count                   8793
recent_trend                           8793
primary_state_name                     8793
cancer                                 8793
race_ethnicity                         8793
sex                                    8793
age                                    8793
age_adjusted_rate                      8793
five_year_trend_rates                  8793
county_state                           8793
index_of_medical_underservice_score    8793
popestimate2015                        8793
popestimate2016                        8793
popestimate2017                        8793
popestimate2018                        8793
popestimate2019                        8793
dtype: int64

In [9]:
#create target 
y = incidence_df[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
0,167.8
1,1738.4
2,339.4
3,2416.0
4,145.2


In [10]:
#create feautures variable holding ages a d mua score
X = incidence_df[["age", "index_of_medical_underservice_score"]]
X.head()

,age,index_of_medical_underservice_score
0,<50,54.0
1,50+,54.0
2,<65,54.0
3,65+,54.0
4,<50,51.0


In [11]:
#create dummy variables for age
X = pd.get_dummies(X, columns=["age"])
X.head()

,index_of_medical_underservice_score,age_50+,age_65+,age_<50,age_<65
0,54.0,0,0,1,0
1,54.0,1,0,0,0
2,54.0,0,0,0,1
3,54.0,0,1,0,0
4,51.0,0,0,1,0


In [12]:
#get shape of target array
y.shape

(8793, 1)

In [13]:
#get shape of feautures array
X.shape

(8793, 5)

In [14]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
#create linear regression model using sklearn
model = LinearRegression()

In [16]:
#fit the model with training data
regression_age_MUA_incidence = model.fit(X_train, y_train)

In [17]:
#Create predictions array using linear regression
y_pred = regression_age_MUA_incidence.predict(X_test)
y_pred

array([[1358.23937863],
       [ 234.83274082],
       [ 111.23559299],
       ...,
       [1965.71898192],
       [1358.35127264],
       [1357.61408858]])

In [91]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9562602889182048

In [18]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [19]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [20]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 4.845e+04
Date:                Sun, 29 May 2022   Prob (F-statistic):               0.00
Time:                        14:59:44   Log-Likelihood:                -57372.
No. Observations:                8793   AIC:                         1.148e+05
Df Residuals:                    8788   BIC:                         1.148e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

In [21]:
#fit the model with training data
#decision tree regression
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [22]:
#predict
y_pred = regressor_DT.predict(X_test)
y_pred

array([1410.9       ,  229.63017241,   85.56666667, ..., 1945.01777778,
       1380.5       , 1354.70689655])

In [23]:
#calculate r2 score
r2_score(y_test, y_pred)

0.9507687392746965